In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0
scikit-learn-intelex==2021.20220215.212715


In [2]:
!python -V

Python 3.9.12


In [3]:
import pickle
import pandas as pd
import os

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename, year, month):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    return df

In [ ]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet', 2023, 3)

: 

In [ ]:
df

In [ ]:
df.duration.describe()

count    3.316216e+06
mean     1.499996e+01
std      1.060465e+01
min      1.000000e+00
25%      7.483333e+00
50%      1.211667e+01
75%      1.930000e+01
max      6.000000e+01
Name: duration, dtype: float64

In [ ]:
def apply_model(output_file):

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred
    
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [ ]:
output_file='predictions_2023_03.parquet'
apply_model(output_file)

: 

In [ ]:
size_bytes = os.path.getsize(output_file)

# Convert to megabytes (optional)
size_mb = size_bytes / (1024 * 1024)

print(f"File size: {size_bytes} bytes ({size_mb:.2f} MB)")

In [ ]:
"!jupyter nbconvert --to script starter.ipynb"